# PyTorch model with 5 modes (for now)
*Step 1:* Collect and label data</br>
*Step 2:* process data and normalize it!</br>
*Step 3:* Convert text to numeric vectors</br>
*Step 4:* Train a Lightweight Classifier</br>
*Step 5:* Evaluate Performance</br>
*Step 6:* Save and Load model to the project</br>

### SQL_mode, insight_mode, Comparison_mode, visualization_mode, prediction_mode


In [13]:
import pandas as pd
import numpy as np
import torch
from sklearn.model_selection import train_test_split

In [14]:
### Prepare the data and join into one df
comparison_df = pd.read_csv("training_data_modes/comparison_mode.csv")
insight_df = pd.read_csv("training_data_modes/insight_mode.csv")
prediction_df = pd.read_csv("training_data_modes/prediction_mode.csv")
sql_df = pd.read_csv("training_data_modes/sql_mode.csv")
visualization_df = pd.read_csv("training_data_modes/visualization_mode.csv")

df = pd.concat([comparison_df, insight_df, prediction_df, sql_df, visualization_df])
X, y = df['text'], df['label']
df.head()

,text,label
0,Which period had better transactions?,comparison_mode
1,Which period had better expenses?,comparison_mode
2,How does conversion rate differ between regions?,comparison_mode
3,How does production costs differ between regions?,comparison_mode
4,Compare customers from this quarter to last qu...,comparison_mode


In [15]:
#### Shuffle the data and split the train test.

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)